In [ ]:
import os
import sys
import shutil
import casadi as cs
import numpy as np
from copy import copy

import torch
from acados_template import AcadosOcp, AcadosOcpSolver, AcadosModel


class DROptimizer:
    def __init__(self, t_horizon, q_cost=None, r_cost=None,
                 
                  model_name="DR_acados_mpc", solver_options=None):
        nx = key_model.x.size()[0]
        nu = key_model.u.size()[0]
        ny = nx + nu
        n_param = key_model.p.size()[0] if isinstance(key_model.p, cs.MX) else 0

        acados_source_path = os.environ['ACADOS_SOURCE_DIR']
        sys.path.insert(0, '../common')

        # Create OCP object to formulate the optimization
        ocp = AcadosOcp()
        ocp.acados_include_path = acados_source_path + '/include'
        ocp.acados_lib_path = acados_source_path + '/lib'
        ocp.model = key_model
        ocp.dims.N = self.N
        ocp.solver_options.tf = t_horizon

        # Initialize parameters
        ocp.dims.np = n_param
        ocp.parameter_values = np.zeros(n_param)

        ocp.cost.cost_type = 'LINEAR_LS'
        ocp.cost.cost_type_e = 'LINEAR_LS'

        ocp.cost.W = np.diag(np.concatenate((q_diagonal, r_cost)))
        ocp.cost.W_e = np.diag(q_diagonal)
        terminal_cost = 0 if solver_options is None or not solver_options["terminal_cost"] else 1
        ocp.cost.W_e *= terminal_cost

        ocp.cost.Vx = np.zeros((ny, nx))
        ocp.cost.Vx[:nx, :nx] = np.eye(nx)
        ocp.cost.Vu = np.zeros((ny, nu))
        ocp.cost.Vu[-4:, -4:] = np.eye(nu)

        ocp.cost.Vx_e = np.eye(nx)

        # Initial reference trajectory (will be overwritten)
        x_ref = np.zeros(nx)
        ocp.cost.yref = np.concatenate((x_ref, np.array([0.0, 0.0, 0.0, 0.0])))
        ocp.cost.yref_e = x_ref

        # Initial state (will be overwritten)
        ocp.constraints.x0 = x_ref

        # Set constraints
        ocp.constraints.lbu = np.array([self.min_u] * 4)
        ocp.constraints.ubu = np.array([self.max_u] * 4)
        ocp.constraints.idxbu = np.array([0, 1, 2, 3])

        # Solver options
        ocp.solver_options.qp_solver = 'FULL_CONDENSING_HPIPM'
        ocp.solver_options.hessian_approx = 'GAUSS_NEWTON'
        ocp.solver_options.integrator_type = 'ERK'
        ocp.solver_options.print_level = 0
        ocp.solver_options.nlp_solver_type = 'SQP_RTI' if solver_options is None else solver_options["solver_type"]

        # Compile acados OCP solver if necessary
        json_file = os.path.join(self.acados_models_dir, key_model.name + '_acados_ocp.json')
        self.acados_ocp_solver[key] = AcadosOcpSolver(ocp, json_file=json_file)


    def clear_acados_model(self):
        """
        Removes previous stored acados models to avoid name conflicts.
        """

        json_file = os.path.join(self.acados_models_dir, 'acados_ocp.json')
        if os.path.exists(json_file):
            os.remove(os.path.join(os.getcwd(), json_file))
        compiled_model_dir = os.path.join(os.getcwd(), 'c_generated_code')
        if os.path.exists(compiled_model_dir):
            shutil.rmtree(compiled_model_dir)


   def acados_setup_model(self, nominal, model_name):
        """
        Builds an Acados symbolic models using CasADi expressions.
        :param model_name: name for the acados model. Must be different from previously used names or there may be
        problems loading the right model.
        :param nominal: CasADi symbolic nominal model of the quadrotor: f(self.x, self.u) = x_dot, dimensions 13x1.
        :return: Returns a total of three outputs, where m is the number of GP's in the GP ensemble, or 1 if no GP:
            - A dictionary of m AcadosModel of the GP-augmented quadrotor
            - A dictionary of m CasADi symbolic nominal dynamics equations with GP mean value augmentations (if with GP)
        :rtype: dict, dict, cs.MX
        """

        def fill_in_acados_model(x, u, p, dynamics, name):

            x_dot = cs.MX.sym('x_dot', dynamics.shape)
            f_impl = x_dot - dynamics

            # Dynamics model
            model = AcadosModel()
            model.f_expl_expr = dynamics
            model.f_impl_expr = f_impl
            model.x = x
            model.xdot = x_dot
            model.u = u
            model.p = p
            model.name = name

            return model

        acados_models = {}
        dynamics_equations = {}

        # Run GP inference if GP's available
        if self.gp_reg_ensemble is not None:

            # Feature vector are the elements of x and u determined by the selection matrix B_z. The trigger var is used
            # to select the gp-specific state estimate in the first optimization node, and the regular integrated state
            # in the rest. The computing of the features z is done within the GPEnsemble.
            gp_x = self.gp_x * self.trigger_var + self.x * (1 - self.trigger_var)
            #  Transform velocity to body frame
            v_b = v_dot_q(gp_x[7:10], quaternion_inverse(gp_x[3:7]))
            gp_x = cs.vertcat(gp_x[:7], v_b, gp_x[10:])
            gp_u = self.u

            gp_dims = self.gp_reg_ensemble.dim_idx

            # Get number of models in GP
            for i in range(self.gp_reg_ensemble.n_models):
                # Evaluate cluster of the GP ensemble
                cluster_id = {k: [v] for k, v in zip(gp_dims, i * np.ones_like(gp_dims, dtype=int))}
                outs = self.gp_reg_ensemble.predict(gp_x, gp_u, return_cov=False, gp_idx=cluster_id, return_z=False)

                # Unpack prediction outputs. Transform back to world reference frame
                outs = self.add_missing_states(outs)
                gp_means = v_dot_q(outs["pred"], gp_x[3:7])
                gp_means = self.remove_extra_states(gp_means)

                # Add GP mean prediction
                dynamics_equations[i] = nominal + cs.mtimes(self.B_x, gp_means)

                x_ = self.x
                dynamics_ = dynamics_equations[i]

                # Add again the gp augmented dynamics for the GP state
                dynamics_ = cs.vertcat(dynamics_)
                dynamics_equations[i] = cs.vertcat(dynamics_equations[i])

                i_name = model_name + "_domain_" + str(i)

                params = cs.vertcat(self.gp_x, self.trigger_var)
                acados_models[i] = fill_in_acados_model(x=x_, u=self.u, p=params, dynamics=dynamics_, name=i_name)

        elif self.mlp_regressor is not None:
            state = self.gp_x * self.trigger_var + self.x * (1 - self.trigger_var)
            #  Transform velocity to body frame
            v_b = v_dot_q(state[7:10], quaternion_inverse(state[3:7]))
            state = cs.vertcat(state[:7], v_b, state[10:])
            mlp_in = v_b

            if self.mlp_conf['torque_output']:
                mlp_in = cs.vertcat(mlp_in, state[10:])

            if self.mlp_conf['u_inp']:
                mlp_in = cs.vertcat(mlp_in, self.u)

            if self.mlp_conf['ground_map_input']:
                map_conf = GroundEffectMapConfig
                map = GroundMapWithBox(np.array(map_conf.box_min),
                                       np.array(map_conf.box_max),
                                       map_conf.box_height,
                                       horizon=map_conf.horizon,
                                       resolution=map_conf.resolution)

                self._map_res = map_conf.resolution

                self._static_ground_map, self._org_to_map_org = map.at(np.array(map_conf.origin))
                ground_map_dx = cs.MX(self._static_ground_map)

                idx = cs.DM(np.arange(0, 3, 1))

                x, y, z = state[0], state[1], state[2]
                orientation = state[3:7]

                x_idxs = cs.floor((x - self._org_to_map_org[0]) / map_conf.resolution) + idx - 1
                y_idxs = cs.floor((y - self._org_to_map_org[1]) / map_conf.resolution) + idx - 1
                ground_patch = ground_map_dx[x_idxs, y_idxs]

                relative_ground_patch = z - ground_patch
                relative_ground_patch = 4 * (cs.fmax(cs.fmin(relative_ground_patch, 0.5), 0.0) - 0.25)

                ground_effect_in = cs.vertcat(cs.reshape(relative_ground_patch, 9, 1), orientation*0)

                mlp_in = cs.vertcat(mlp_in, ground_effect_in)

            if not self.mlp_conf['approximated']:
                outs = self.mlp_regressor(mlp_in)
            else:
                outs = self.mlp_regressor.approx(mlp_in, order=self.mlp_conf['approx_order'], parallel=False)

            if self.mlp_conf['torque_output']:
                outs_force = outs[:3]
                outs_torque = outs[3:]
                mlp_means = cs.vertcat(v_dot_q(outs_force, state[3:7]), outs_torque)
            else:
                # Unpack prediction outputs. Transform back to world reference frame
                mlp_means = v_dot_q(outs, state[3:7])

            # Add GP mean prediction
            dynamics_equations[0] = nominal + cs.mtimes(self.B_x, mlp_means)

            x_ = self.x
            dynamics_ = dynamics_equations[0]

            # Add again the gp augmented dynamics for the GP state
            dynamics_ = cs.vertcat(dynamics_)
            dynamics_equations[0] = cs.vertcat(dynamics_equations[0])

            i_name = model_name + "_domain_" + str(0)

            if not self.mlp_conf['approximated']:
                params = cs.vertcat(self.gp_x, self.trigger_var)
            else:
                params = cs.vertcat(self.gp_x, self.trigger_var,
                                    self.mlp_regressor.sym_approx_params(order=self.mlp_conf['approx_order'],
                                                                         flat=True))
            acados_models[0] = fill_in_acados_model(x=x_, u=self.u, p=params, dynamics=dynamics_, name=i_name)

        else:

            # No available GP so return nominal dynamics
            dynamics_equations[0] = nominal

            x_ = self.x
            dynamics_ = nominal

            acados_models[0] = fill_in_acados_model(x=x_, u=self.u, p=[], dynamics=dynamics_, name=model_name)

        return acados_models, dynamics_equations


    def run_optimization(self, initial_state=None, use_model=0, return_x=False, gp_regression_state=None):
        """
        Optimizes a trajectory to reach the pre-set target state, starting from the input initial state, that minimizes
        the quadratic cost function and respects the constraints of the system

        :param initial_state: 13-element list of the initial state. If None, 0 state will be used
        :param use_model: integer, select which model to use from the available options.
        :param return_x: bool, whether to also return the optimized sequence of states alongside with the controls.
        :param gp_regression_state: 13-element list of state for GP prediction. If None, initial_state will be used.
        :return: optimized control input sequence (flattened)
        """

        if initial_state is None:
            initial_state = [0, 0, 0] + [1, 0, 0, 0] + [0, 0, 0] + [0, 0, 0]

        # Set initial state. Add gp state if needed
        x_init = initial_state
        x_init = np.stack(x_init)

        # Set initial condition, equality constraint
        self.acados_ocp_solver[use_model].set(0, 'lbx', x_init)
        self.acados_ocp_solver[use_model].set(0, 'ubx', x_init)

        # Set parameters
        if self.with_gp:
            gp_state = gp_regression_state if gp_regression_state is not None else initial_state
            self.acados_ocp_solver[use_model].set(0, 'p', np.array(gp_state + [1]))
            for j in range(1, self.N):
                self.acados_ocp_solver[use_model].set(j, 'p', np.array([0.0] * (len(gp_state) + 1)))

        if self.with_mlp:
            if self.x_opt_acados is None:
                if isinstance(self.target[0], list):
                    self.x_opt_acados = np.expand_dims(
                        np.concatenate([self.target[i] for i in range(len(self.target))]), 0)
                    self.x_opt_acados = self.x_opt_acados.repeat(self.N, 0)
                else:
                    self.x_opt_acados = np.hstack(self.target)
            if self.w_opt_acados is None:
                if len(self.u_target.shape) == 1:
                    self.w_opt_acados = self.u_target[np.newaxis]
                    self.w_opt_acados = self.w_opt_acados.repeat(self.N, 0)
                else:
                    self.w_opt_acados = np.hstack(self.u_target)

            gp_state = gp_regression_state if gp_regression_state is not None else initial_state
            if not self.mlp_conf['approximated']:
                self.acados_ocp_solver[use_model].set(0, 'p', np.hstack([np.array(gp_state + [1])]))
                for j in range(1, self.N):
                    self.acados_ocp_solver[use_model].set(j, 'p', np.hstack([np.array([0.0] * (len(gp_state) + 1))]))
            else:
                state = np.vstack([np.array([initial_state]), self.x_opt_acados[1:]])
                a_list = []
                for i in range(state.shape[0]):
                    a_list.append(v_dot_q(np.array(state[i, 7:10]), quaternion_inverse(np.array(state[i, 3:7]))))
                a = np.array(a_list)[:self.N]

                if self.mlp_conf['torque_output']:
                    a = np.concatenate([a, state[:self.N, 10:]], axis=-1)

                if self.mlp_conf['u_inp']:
                    a = np.concatenate([a, self.w_opt_acados], axis=-1)

                if self.mlp_conf['ground_map_input']:
                    ground_maps = []
                    for i in range(state.shape[0]):
                        pos = state[i][:3]
                        x_idxs = np.floor((pos[0] - self._org_to_map_org[0]) / self._map_res).astype(int) - 1
                        y_idxs = np.floor((pos[1] - self._org_to_map_org[1]) / self._map_res).astype(int) - 1
                        ground_patch = self._static_ground_map[x_idxs:x_idxs + 3, y_idxs:y_idxs + 3]

                        relative_ground_patch = 4 * (np.clip(pos[2] - ground_patch, 0, 0.5) - 0.25)

                        flatten_relative_ground_patch = relative_ground_patch.flatten(order='F')

                        ground_effect_in = np.hstack([flatten_relative_ground_patch,
                                                      flatten_relative_ground_patch[..., :4] * 0])

                        ground_maps.append(ground_effect_in)

                    a = np.concatenate([a, np.array(ground_maps)[:self.N]], axis=-1)

                mlp_params = self.mlp_regressor.approx_params(a, order=self.mlp_conf['approx_order'], flat=True)
                mlp_params = np.vstack([mlp_params, mlp_params[[-1]]])
                self.acados_ocp_solver[use_model].set(0, 'p',
                                                      np.hstack([np.array(gp_state + [1]), mlp_params[0]]))
                for j in range(1, self.N):
                    self.acados_ocp_solver[use_model].set(j, 'p', np.hstack([np.array([0.0] * (len(gp_state) + 1)),
                                                                             mlp_params[j]]))

        # Solve OCP
        self.acados_ocp_solver[use_model].solve()

        # Get u
        w_opt_acados = np.ndarray((self.N, 4))
        x_opt_acados = np.ndarray((self.N + 1, len(x_init)))
        x_opt_acados[0, :] = self.acados_ocp_solver[use_model].get(0, "x")
        for i in range(self.N):
            w_opt_acados[i, :] = self.acados_ocp_solver[use_model].get(i, "u")
            x_opt_acados[i + 1, :] = self.acados_ocp_solver[use_model].get(i + 1, "x")

        self.x_opt_acados = x_opt_acados.copy()
        self.w_opt_acados = w_opt_acados.copy()

        w_opt_acados = np.reshape(w_opt_acados, (-1))
        return w_opt_acados if not return_x else (w_opt_acados, x_opt_acados)




In [ ]:
import casadi as cs
import numpy as np
import torch
import torch.nn as nn
import l4casadi as l4c
from acados_template import AcadosSimSolver, AcadosOcpSolver, AcadosSim, AcadosOcp, AcadosModel
import time
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'



class DeepGRURegressor0527(nn.Module):
    def __init__(
        self,
        input_size: int = 1,
        hidden_size: int = 64,
        num_layers: int = 6,
        dropout: float = 0.3,
        bidirectional: bool = False,
        fc_hidden_dims: list[int] = [512, 256, 64, 32, 32]
    ):
        """
        A deep GRU-based regressor with configurable GRU depth, directionality,
        and an MLP head of arbitrary hidden dimensions.
        
        Args:
            input_size:    Number of features in the input sequence.
            hidden_size:   Number of features in the GRU hidden state.
            num_layers:    Number of stacked GRU layers.
            dropout:       Dropout probability between GRU layers and in MLP.
            bidirectional: If True, uses a bidirectional GRU (doubles hidden output).
            fc_hidden_dims: List of hidden sizes for the MLP head.
        """
        super().__init__()
        self.bidirectional = bidirectional
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
            bidirectional=bidirectional,
        )

        # Determine the input dimension to the first FC layer
        fc_in_dim = hidden_size * (2 if bidirectional else 1)

        # Build MLP head
        layers = []
        for h_dim in fc_hidden_dims:
            layers += [
                nn.Linear(fc_in_dim, h_dim),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout),
            ]
            fc_in_dim = h_dim
        layers.append(nn.Linear(fc_in_dim, 1))

        self.fc = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor, hidden:torch.Tensor = None ) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Forward pass.
        Args:
            x: Tensor of shape (batch, seq_len, input_size)
        Returns:
            preds: Tensor of shape (batch,) with the regression output.
            hidden: (num_layers, batch, hidden_size) or None
        """
        # GRU returns: output (batch, seq_len, num_directions*hidden_size), h_n
        B = x.size(0)
        if hidden is not None and hidden.size(1) == B:
            hidden = hidden.detach()
        else:
            hidden = None
        out,_ = self.gru(x)
        y_seq = self.fc(out)         # (B, seq_len, 1)
        y_seq = y_seq.squeeze(-1)    # (B, seq_len)
        return y_seq, hidden

class MPC:
    def __init__(self, model, N):
        self.N = N
        self.model = model

    @property
    def solver(self):
        return AcadosOcpSolver(self.ocp())

    def ocp(self):
        model = self.model



def run():

    # -------- Load the previously trained GRU model --------------------------------------
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    save_path = os.path.join("save", "gru_regressor_0527.pth")
    model_gru = DeepGRURegressor0527()                                      # 1) re-create the model with the same architecture
    model_gru.load_state_dict(torch.load(save_path, map_location=device))   # 2) load weights   
    model_gru.to(device).eval()                                             # 3) move to device & eval
    print("Model reloaded and ready for inference on", device)

    MPC_Horizon = 10

    model_CasADi = l4c.realtime.RealTimeL4CasADi(model_gru, approximation_order=2)

    solver = MPC(model=model_CasADi.model(), MPC_Horizon = MPC_Horizon).solver


    x = []
    x_ref = []
    ts = 1. / N
    xt = np.array([1., 0.])
    opt_times = []

    for i in range(50):
        now = time.time()
        t = np.linspace(i * ts, i * ts + 1., 10)
        yref = np.sin(0.5 * t + np.pi / 2)

if __name__ == '__main__':
    run()

/tmp/ipykernel_352256/627214486.py:269: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_gru.load_state_dict(torch.load(save_path, map_location=device))   # 2) load weigh

Model reloaded and ready for inference on cuda


TypeError: DeepGRURegressor0527.forward() missing 1 required positional argument: 'x'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
save_path = os.path.join("save", "gru_regressor_0527.pth")
model_gru = DeepGRURegressor0527()                                      # 1) re-create the model with the same architecture
model_gru.load_state_dict(torch.load(save_path, map_location=device))   # 2) load weights   
model_gru.to(device).eval()                                             # 3) move to device & eval
print("Model reloaded and ready for inference on", device)

MPC_Horizon = 10

model_CasADi = l4c.realtime.RealTimeL4CasADi(model_gru, approximation_order=2)

Model reloaded and ready for inference on cuda


/tmp/ipykernel_352256/1129356520.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_gru.load_state_dict(torch.load(save_path, map_location=device))   # 2) load weight

In [ ]:
import casadi as cs
import numpy as np
import torch
import l4casadi as l4c
from acados_template import AcadosSimSolver, AcadosOcpSolver, AcadosSim, AcadosOcp, AcadosModel
import time
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'


class PyTorchModel(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.input_layer = torch.nn.Linear(2, 512)

        hidden_layers = []
        for i in range(5):
            hidden_layers.append(torch.nn.Linear(512, 512))

        self.hidden_layer = torch.nn.ModuleList(hidden_layers)
        self.out_layer = torch.nn.Linear(512, 2)

        # Model is not trained -- setting output to zero
        with torch.no_grad():
            self.out_layer.bias.fill_(0.)
            self.out_layer.weight.fill_(0.)

    def forward(self, x):
        x = self.input_layer(x)
        for layer in self.hidden_layer:
            x = torch.tanh(layer(x))
        x = self.out_layer(x)
        return x


class DoubleIntegratorWithLearnedDynamics:
    def __init__(self, learned_dyn):
        self.learned_dyn = learned_dyn

    def model(self):
        s = cs.MX.sym('s', 1)
        s_dot = cs.MX.sym('s_dot', 1)
        s_dot_dot = cs.MX.sym('s_dot_dot', 1)
        u = cs.MX.sym('u', 1)
        x = cs.vertcat(s, s_dot)
        x_dot = cs.vertcat(s_dot, s_dot_dot)

        res_model = self.learned_dyn(x)
        p = self.learned_dyn.get_sym_params()
        parameter_values = self.learned_dyn.get_params(np.array([0, 0]))

        f_expl = cs.vertcat(
            s_dot,
            u
        ) + res_model

        x_start = np.zeros((2))

        # store to struct
        model = cs.types.SimpleNamespace()
        model.x = x
        model.xdot = x_dot
        model.u = u
        model.z = cs.vertcat([])
        model.p = p
        model.parameter_values = parameter_values
        model.f_expl = f_expl
        model.x_start = x_start
        model.constraints = cs.vertcat([])
        model.name = "wr"

        return model


class MPC:
    def __init__(self, model, N):
        self.N = N
        self.model = model

    @property
    def simulator(self):
        return AcadosSimSolver(self.sim())

    @property
    def solver(self):
        return AcadosOcpSolver(self.ocp())

    def sim(self):
        model = self.model

        t_horizon = 1.
        N = self.N

        # Get model
        model_ac = self.acados_model(model=model)
        model_ac.p = model.p

        # Dimensions
        nx = 2
        nu = 1
        ny = 1

        # Create OCP object to formulate the optimization
        sim = AcadosSim()
        sim.model = model_ac
        sim.dims.N = N
        sim.dims.nx = nx
        sim.dims.nu = nu
        sim.dims.ny = ny
        sim.solver_options.tf = t_horizon

        # Solver options
        sim.solver_options.Tsim = 1./ 10.
        sim.solver_options.qp_solver = 'FULL_CONDENSING_HPIPM'
        sim.solver_options.hessian_approx = 'GAUSS_NEWTON'
        sim.solver_options.integrator_type = 'ERK'
        # ocp.solver_options.print_level = 0
        sim.solver_options.nlp_solver_type = 'SQP_RTI'

        return sim

    def ocp(self):
        model = self.model

        t_horizon = 1.
        N = self.N

        # Get model
        model_ac = self.acados_model(model=model)
        model_ac.p = model.p

        # Dimensions
        nx = 2
        nu = 1
        ny = 1

        # Create OCP object to formulate the optimization
        ocp = AcadosOcp()
        ocp.model = model_ac
        ocp.dims.N = N
        ocp.dims.nx = nx
        ocp.dims.nu = nu
        ocp.dims.ny = ny
        ocp.solver_options.tf = t_horizon

        # Initialize cost function
        ocp.cost.cost_type = 'LINEAR_LS'
        ocp.cost.cost_type_e = 'LINEAR_LS'

        ocp.cost.W = np.array([[1.]])

        ocp.cost.Vx = np.zeros((ny, nx))
        ocp.cost.Vx[0, 0] = 1.
        ocp.cost.Vu = np.zeros((ny, nu))
        ocp.cost.Vz = np.array([[]])
        ocp.cost.Vx_e = np.zeros((ny, nx))
        ocp.cost.W_e = np.array([[0.]])
        ocp.cost.yref_e = np.array([0.])

        # Initial reference trajectory (will be overwritten)
        ocp.cost.yref = np.zeros(1)

        # Initial state (will be overwritten)
        ocp.constraints.x0 = model.x_start

        # Set constraints
        a_max = 10
        ocp.constraints.lbu = np.array([-a_max])
        ocp.constraints.ubu = np.array([a_max])
        ocp.constraints.idxbu = np.array([0])

        # Solver options
        ocp.solver_options.qp_solver = 'FULL_CONDENSING_HPIPM'
        ocp.solver_options.hessian_approx = 'GAUSS_NEWTON'
        ocp.solver_options.integrator_type = 'ERK'
        ocp.solver_options.nlp_solver_type = 'SQP_RTI'

        ocp.parameter_values = model.parameter_values

        return ocp

    def acados_model(self, model):
        model_ac = AcadosModel()
        model_ac.f_impl_expr = model.xdot - model.f_expl
        model_ac.f_expl_expr = model.f_expl
        model_ac.x = model.x
        model_ac.xdot = model.xdot
        model_ac.u = model.u
        model_ac.name = model.name
        return model_ac


def run():
    N = 10

    learned_dyn_model = l4c.realtime.RealTimeL4CasADi(PyTorchModel(), approximation_order=1)

    model = DoubleIntegratorWithLearnedDynamics(learned_dyn_model)
    solver = MPC(model=model.model(), N=N).solver

    print('Warming up model...')
    x_l = []
    for i in range(N):
        x_l.append(solver.get(i, "x"))
    for i in range(20):
        learned_dyn_model.get_params(np.stack(x_l, axis=0))
    print('Warmed up!')

    x = []
    x_ref = []
    ts = 1. / N
    xt = np.array([1., 0.])
    opt_times = []

    for i in range(50):
        now = time.time()
        t = np.linspace(i * ts, i * ts + 1., 10)
        yref = np.sin(0.5 * t + np.pi / 2)
        x_ref.append(yref[0])
        for t, ref in enumerate(yref):
            solver.set(t, "yref", ref)
        solver.set(0, "lbx", xt)
        solver.set(0, "ubx", xt)
        solver.solve()
        xt = solver.get(1, "x")
        x.append(xt)

        x_l = []
        for i in range(N):
            x_l.append(solver.get(i, "x"))
        params = learned_dyn_model.get_params(np.stack(x_l, axis=0))
        for i in range(N):
            solver.set(i, "p", params[i])

        elapsed = time.time() - now
        opt_times.append(elapsed)

    print(f'Mean iteration time: {1000*np.mean(opt_times):.1f}ms -- {1/np.mean(opt_times):.0f}Hz)')


if __name__ == '__main__':
    run()

In [ ]:
# ─── PyTorch GRU REGRESSOR DEFINITION & LOADING ────────────────────────────────
class GRURegressor(nn.Module):
    def __init__(self, input_size=1, hidden_size=128, num_layers=2, output_size=1):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers,
                          batch_first=True)
        self.fc  = nn.Linear(hidden_size, output_size)

    def forward(self, x, h=None):
        # x: [batch, seq_len, input_size]
        out, h = self.gru(x, h)           # out: [batch, seq_len, hidden_size]
        out = self.fc(out)                # out: [batch, seq_len, output_size]
        return out, h

def load_model(path, device):
    # instantiate model with the exact same hyperparameters as training
    model = GRURegressor(input_size=1,
                         hidden_size=128,
                         num_layers=2,
                         output_size=1).to(device)
    state = torch.load(path, map_location=device)
    # if you saved state_dict:
    model.load_state_dict(state)
    model.eval()
    return model

# ─── SIMPLE RANDOM-SHOOTING MPC FUNCTION ───────────────────────────────────────

def mpc_step(model, elapsed_time, ref_time, ref_speed,
             horizon=10, num_candidates=30,
             u_min=-15.0, u_max=100.0, Ts=0.01, device='cpu'):
    """
    Random-shooting MPC:
      • samples `num_candidates` PWM sequences of length `horizon`
      • rolls them through the GRU model to predict speeds
      • computes sum-of-squared tracking error against the reference
      • returns the first control move of the best sequence.
    """
    # 1) build future reference vector
    ref_future = np.array([
        float(np.interp(elapsed_time + Ts*(i+1), ref_time, ref_speed))
        for i in range(horizon)
    ], dtype=np.float32)  # shape (horizon,)

    # 2) sample candidate sequences in [u_min, u_max]
    cands = np.random.uniform(u_min, u_max,
                              size=(num_candidates, horizon)).astype(np.float32)

    # 3) roll out each candidate, compute cost
    costs = np.zeros(num_candidates, dtype=np.float32)
    with torch.no_grad():
        for i in range(num_candidates):
            u_seq = torch.from_numpy(cands[i:i+1, :]) \
                       .unsqueeze(2).to(device)  # [1, horizon, 1]
            preds, _ = model(u_seq)    # [1, horizon, 1]
            v_pred = preds.squeeze(0).squeeze(1).cpu().numpy()  # (horizon,)
            costs[i] = np.sum((v_pred - ref_future)**2)
    # 4) pick best and return its first element
    best_idx = int(np.argmin(costs))
    return float(cands[best_idx, 0])

def mpc_control(past_u, ref_times, ref_speeds, elapsed_time, Ts, model, Np):
    """
    Simple receding-horizon MPC: assume model takes input sequence of length Np and predicts Np future speeds.
    We optimize the first control move by fitting a constant-u sequence over horizon.
    """
    # future reference
    t_future = elapsed_time + np.arange(1, Np+1) * Ts
    r_future = np.interp(t_future, ref_times, ref_speeds)

    def cost(u_seq):
        # prepare input: shape [1, Np, 1]
        u_in = torch.tensor(u_seq[None, :, None], dtype=torch.float32, device=device)
        with torch.no_grad():
            v_pred = model(u_in).cpu().numpy().flatten()
        return np.sum((v_pred - r_future)**2)

    # init guess: keep previous input
    u0 = np.ones(Np, dtype=float) * past_u[-1]
    bounds = [(-15.0, 100.0)] * Np
    res = minimize(cost, u0, bounds=bounds, options={'maxiter': 10, 'disp': False})
    if res.success:
        return float(res.x[0])
    else:
        return float(u0[0])

